In [4]:
import pandas as pd
import numpy as np
from My_numpy import Array, Null
from prettytable import PrettyTable
import csv


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import My_pandas as mpd

In [6]:
!pwd

/Users/ila/Desktop/Python/ООП


In [7]:
class Dop_str_for_DataFrame(str):
    def __repr__(self):
        return (self)

In [8]:
type(list)

type

In [20]:
class ft_Logistic_Regression():
    exp = 2.7182818284

    def __init__(self, lr=0.0001, n_epochs=1000, min_err=0.02, n_clas=2, penalty=None, penalty_param=0.5,
                 stochastic=False, chunk_size=5):
        self.lr = lr
        self.n_epochs = n_epochs
        self.min_err = min_err
        self.n_clas = n_clas
        self.penalty_param = penalty_param
        self.stochastic = stochastic
        self.chunk_size = chunk_size
        if penalty == 'l1':
            self.penalty = self.__l1_penalty__
        elif penalty == 'l2':
            self.penalty = self.__l2_penalty__
        else:
            self.penalty = self.__none_penalty__
        if self.n_clas != 2:
            self.multiflag = True
        else:
            self.multiflag = False

    def sign(self, n):
        if isinstance(n, (int, float)):
            if n >= 0:
                return 1
            return -1
        else:
            return mnp.Array([1 if i >= 0 else -1 for i in n])

    def __l1_penalty__(self):
        return self.penalty_param * self.sign(self.w_mas)

    def __l2_penalty__(self):
        return [w * self.penalty_param for w in self.w_mas]  # * 2

    def __none_penalty__(self):
        return 0

    def regression(self, data, w_mas):  # Линейный алгоритм
        sums = []
        for row in data.data[1:, 1:]:
            sums += [(row * w_mas).sum()]
        return sums

    def sigmoid(self, regs):
        sigmoids = []
        for reg in regs:
            if reg > 100:
                reg = 100
            elif reg < -100:
                reg = -100
            sigmoids += [1 / (1 + self.exp ** -(reg))]
        return mnp.Array(sigmoids)

    def Logloss(self, sigs, targets):
        sums = 0
        eps = 1e-15
        for sig, target in zip(sigs, targets):
            if sig == 1 or sig == 0:
                sig = max(eps, min(1 - eps, sig))
            res = -(target * np.log(sig) + (1 - target) * np.log(1 - sig))
            sums += res
        return sums / len(sigs)

    def derivative(self, sigs, targets, data):
        der_w = mnp.Array([])
        rasn_sig_tar = sigs - targets
        for column in data.columns[1:]:
            der_w.append(mnp.Array(rasn_sig_tar * data[column]).mean())
        return der_w

    def multiclass_fit(self, data, targets):
        self.multiflag = False
        self.dict_multiclass = {}
        for target in set(targets):
            copy_target = [1 if i == target else 0 for i in targets]
            self.fit(data, copy_target)
            self.dict_multiclass[target] = self.w_mas
        self.multiflag = True

    def argmx_dict(self, dicts):
        mas_target = []
        for i in range(len(list(dicts.values())[0])):
            max_int = float('-inf')
            for key in dicts.keys():
                if dicts[key][i] > max_int:
                    target = key
                    max_int = dicts[key][i]
            mas_target.append(target)
        return mas_target

    def multiclass_predict(self, data):
        self.multiflag = False
        d = {}
        for key in self.dict_multiclass.keys():
            d[key] = self.predict_proba(data, w_mas=self.dict_multiclass[key])
        self.multiflag = True
        return self.argmx_dict(d)

    def fit(self, data, target):
        if self.multiflag == True:
            return self.multiclass_fit(data, target)

        data['dop_par'] = [1] * data.shape[0]
        self.w_mas = [0] * (data.shape[1])
        self.mas_logloss = []
        self.mas_w_er = [0]

        for epoch in range(self.n_epochs):
            reg = self.regression(data, self.w_mas)
            sig = self.sigmoid(reg)

            if self.stochastic == True:
                rand_st = np.random.randint(data.shape[0] - self.chunk_size)
                rand_fn = rand_st + self.chunk_size
                der_w = self.derivative(sig[rand_st:rand_fn], target[rand_st:rand_fn], train_data.iloc(rand_st, rand_fn + 1))
                self.w_mas = self.w_mas - self.lr * (der_w + self.penalty())
            else:
                der_w = self.derivative(sig, target, data)
                self.w_mas = self.w_mas - self.lr * (der_w + self.penalty())

            log = self.Logloss(sig, target)
            self.mas_logloss.append(log)
            self.mas_w_er.append(self.w_mas[1])
            if log < self.min_err:
                break
            if epoch > 5:
                a = abs(self.mas_logloss[epoch - 1] - self.mas_logloss[epoch])
                b = abs(self.mas_logloss[epoch - 2] - self.mas_logloss[epoch - 3])
                if abs(a - b) < self.min_err:
                    break

        data = data.drop('dop_par')

    def predict_proba(self, data, multiclass=None, w_mas=None):
        if self.multiflag == True:
            return self.multiclass_predict(data)
        if w_mas == None:
            w_mas = self.w_mas
        data['dop_par'] = [1] * data.shape[0]

        reg = self.regression(data, w_mas)
        sig = self.sigmoid(reg)

        data = data.drop('dop_par')
        self.w_mas = w_mas
        return sig

    def predict(self, data, okrugl=0.5):
        sig = self.predict_proba(data)
        return [1 if i >= okrugl else 0 for i in sig]

In [10]:
def read_csv(file_name, encoding='UTF-8', mode='r', delimiter=',', quotechar='|'):
    if file_name[-4:] != '.csv':
        file_name += '.csv'
    with open(file_name, encoding=encoding, mode=mode) as file:
        file_csv = csv.reader(file, delimiter=delimiter, quotechar=quotechar)
        all_mas = []
        for en, i in enumerate(file_csv):
            if en == 0:
                columns_name = i
            else:
                mas = i
                for j in range(len(mas)):
                    try:
                        if mas[j] == '':
                            mas[j] = Null()
                        else:
                            mas[j] = float(mas[j])
                    except:
                        mas[j]
                all_mas += [mas]
        return DataFrame(all_mas, columns=columns_name)

In [37]:
np.exp(1000000)

<ipython-input-37-3e16d55340bd>:1: RuntimeWarning: overflow encountered in exp
  np.exp(1000000)


inf

In [38]:
exp = 2.7182818284
exp ** -100

3.7200759841014264e-44

# проблемы 

1. будет проблема, если имена колонок не уникальны--------- а с другой стороны, это даже хорошо, не баг а фича 
2. добавить в read_csv параметр index_col ----------- способ решения
3. функцции sorted и round могут быть запрещены

##  Тестирование iloc

In [30]:
a = DataFrame([[4,5,6], [7,8,9], [10,11,12], [13,14,15]], columns=['1', '2', '3'])
a

+-------+----+----+----+
| index | 1  | 2  | 3  |
+-------+----+----+----+
|   0   | 4  | 5  | 6  |
|   1   | 7  | 8  | 9  |
|   2   | 10 | 11 | 12 |
|   3   | 13 | 14 | 15 |
+-------+----+----+----+

In [25]:
a.iloc(1)

+-------+---+---+---+
| index | 0 | 1 | 2 |
+-------+---+---+---+
|   0   | 7 | 8 | 9 |
+-------+---+---+---+

In [35]:
a.iloc(1, 1+2+1)

+-------+----+----+----+
| index | 0  | 1  | 2  |
+-------+----+----+----+
|   0   | 7  | 8  | 9  |
|   1   | 10 | 11 | 12 |
|   2   | 13 | 14 | 15 |
+-------+----+----+----+

## Тестирование fillna()

In [47]:
a = DataFrame([[1,2,3], [1,2]])
a

+-------+---+---+------+
| index | 0 | 1 |  2   |
+-------+---+---+------+
|   0   | 1 | 2 |  3   |
|   1   | 1 | 2 | Null |
+-------+---+---+------+

In [48]:
a.fillna(0)

+-------+---+---+---+
| index | 0 | 1 | 2 |
+-------+---+---+---+
|   0   | 1 | 2 | 3 |
|   1   | 1 | 2 | 0 |
+-------+---+---+---+

In [52]:
a.norm()
a

+-------+-----+-----+------+
| index |  0  |  1  |  2   |
+-------+-----+-----+------+
|   0   | 0.0 | 0.0 | 1.0  |
|   1   | 0.0 | 0.0 | -1.0 |
+-------+-----+-----+------+

## Тестирование drop

In [8]:
a = DataFrame([[1,2,3], [1,2,3]], columns=['1', '2', '3'])
a

+-------+---+---+---+
| index | 1 | 2 | 3 |
+-------+---+---+---+
|   0   | 1 | 2 | 3 |
|   1   | 1 | 2 | 3 |
+-------+---+---+---+

In [9]:
a.drop('3').drop('1')

+-------+---+
| index | 2 |
+-------+---+
|   0   | 2 |
|   1   | 2 |
+-------+---+

In [198]:
a.data

[['index', '1', '2', '3'], [0, 1, 2, 3], [1, 1, 2, 3]]

In [200]:
a['1']

[1, 1]

## Тестирование None

In [8]:
pd.DataFrame([[1,2,3], [1,2]])

,0,1,2
0,1,2,3.0
1,1,2,NaN


In [9]:
a = DataFrame([[1,2,3], [1,2]])
a

+-------+---+---+------+
| index | 0 | 1 |  2   |
+-------+---+---+------+
|   0   | 1 | 2 |  3   |
|   1   | 1 | 2 | Null |
+-------+---+---+------+

## Тестирование percentile

In [10]:
def percentile(data, percen):
    data = list(set(data))
    index = int(len(data) * percen / 100 + 1)
    sort_data = sorted(data)
    return sort_data[index - 1]

In [11]:
l = [2,4,8,9,11,13,15,17,20,24,29,30]

In [12]:
percentile(l, 10)

4

## Тестирование describe


In [10]:
path = '/Users/ila/Desktop/Python/my projects/logistic_regression/'
mdata = read_csv(path + "resources/dataset_train.csv")
data = pd.read_csv(path + "resources/dataset_train.csv")

In [11]:
mdata.head(5, types = '1')

+-------+-------+----------------+------------+-----------+------------+-----------+------------+-----------+-----------+-------------------------------+------------+----------------+---------------+------------------+-----------------+---------+---------------------------+----------+---------+
| index | Index | Hogwarts House | First Name | Last Name |  Birthday  | Best Hand | Arithmancy | Astronomy | Herbology | Defense Against the Dark Arts | Divination | Muggle Studies | Ancient Runes | History of Magic | Transfiguration | Potions | Care of Magical Creatures |  Charms  |  Flying |
+-------+-------+----------------+------------+-----------+------------+-----------+------------+-----------+-----------+-------------------------------+------------+----------------+---------------+------------------+-----------------+---------+---------------------------+----------+---------+
|   0   |  0.0  |   Ravenclaw    |   Tamara   |    Hsu    | 2000-03-30 |    Left   |  58384.0   |  -487.886 |   

In [12]:
data.describe()

,Index,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
count,1600.00000,1566.000000,1568.000000,1567.000000,1569.000000,1561.000000,1565.000000,1565.000000,1557.000000,1566.000000,1570.000000,1560.000000,1600.000000,1600.000000
mean,799.50000,49634.570243,39.797131,1.141020,-0.387863,3.153910,-224.589915,495.747970,2.963095,1030.096946,5.950373,-0.053427,-243.374409,21.958012
std,462.02453,16679.806036,520.298268,5.219682,5.212794,4.155301,486.344840,106.285165,4.425775,44.125116,3.147854,0.971457,8.783640,97.631602
min,0.00000,-24370.000000,-966.740546,-10.295663,-10.162119,-8.727000,-1086.496835,283.869609,-8.858993,906.627320,-4.697484,-3.313676,-261.048920,-181.470000
25%,399.75000,38511.500000,-489.551387,-4.308182,-5.259095,3.099000,-577.580096,397.511047,2.218653,1026.209993,3.646785,-0.671606,-250.652600,-41.870000
50%,799.50000,49013.500000,260.289446,3.469012,-2.589342,4.624000,-419.164294,463.918305,4.378176,1045.506996,5.874837,-0.044811,-244.867765,-2.515000
75%,1199.25000,60811.250000,524.771949,5.419183,4.904680,5.667000,254.994857,597.492230,5.825242,1058.436410,8.248173,0.589919,-232.552305,50.560000
max,1599.00000,104956.000000,1016.211940,11.612895,9.667405,10.032000,1092.388611,745.396220,11.889713,1098.958201,13.536762,3.056546,-225.428140,279.070000


In [13]:
print(mdata.describe(types = '2'))

+-------------------------------+-------+----------+----------+-----------+----------+----------+----------+----------+
|            Features           | Count |   Mean   |   Std    |    Min    |   25%    |   50%    |   75%    |   Max    |
+-------------------------------+-------+----------+----------+-----------+----------+----------+----------+----------+
|             index             |  1600 |  799.5   |  461.88  |     0     |   400    |   800    |   1200   |   1599   |
|             Index             |  1600 |  799.5   |  461.88  |    0.0    |  400.0   |  800.0   |  1200.0  |  1599.0  |
|           Arithmancy          |  1566 | 49634.57 | 16674.48 |  -24370.0 | 38434.0  | 49018.0  | 60918.0  | 104956.0 |
|           Astronomy           |  1568 |  39.797  | 520.132  |  -966.741 | -489.494 | 261.645  |  525.91  | 1016.212 |
|           Herbology           |  1567 |  1.141   |  5.218   |  -10.296  |  -4.312  |  3.469   |  5.421   |  11.613  |
| Defense Against the Dark Arts |  1569 

In [17]:
# print(mdata.describe())

## Тестирование read_csv

оригинальный read_csv должен пытаться сразу перегнать все данный в float

In [18]:
path = '/Users/ila/Desktop/Python/my projects/logistic_regression/'
mdata = read_csv(path + "resources/dataset_train.csv")
data = pd.read_csv(path + "resources/dataset_train.csv")

In [19]:
flag = True
for i in data.columns:
    if not type(data[i][0]) == str:
        if not round(data[i][0], 4) == round(mdata[i][0], 4):
            flag = False
            break
    else:
        if not data[i][0] == mdata[i][0]:
            flag = False
            break
        
if flag == True:
    print('Ошибок нет')
else:
    print('Ошибка в', i )

Ошибок нет


параметр  index_col

In [20]:
path = '/Users/ila/Desktop/Python/my projects/logistic_regression/'
mdata = read_csv(path + "resources/dataset_train.csv")
data = pd.read_csv(path + "resources/dataset_train.csv", index_col='Last Name')

In [21]:
mdata.head(types = 2)

+-------------------------------+---------------------+--------------------+---------------------+-----------------------+---------------------+
|            Field 1            |       Field 2       |      Field 3       |       Field 4       |        Field 5        |       Field 6       |
+-------------------------------+---------------------+--------------------+---------------------+-----------------------+---------------------+
|             index             |          0          |         1          |          2          |           3           |          4          |
|             Index             |         0.0         |        1.0         |         2.0         |          3.0          |         4.0         |
|         Hogwarts House        |      Ravenclaw      |     Slytherin      |      Ravenclaw      |       Gryffindor      |      Gryffindor     |
|           First Name          |        Tamara       |       Erich        |       Stephany      |         Vesta         |        

In [22]:
mdata['Transfiguration'][0]

1039.7882807428462

## Тестирование слайсинга

In [23]:
list1 = [['1', '1.1', 'a', 3], ['2', '2.2', 'b',6], ['3','3.3', 'c',9]]
a = DataFrame(list1)
b = pd.DataFrame(list1)


In [24]:
for i in range(len(list1)):
    if not all([[all(b[i] == a[i])], [b[i][0] == a[i][0]]]):
        print('ошибка')
print('Ошибок нет')

Ошибок нет


## Тестирование 

In [25]:
a = [[0, 10, 'bmv', 2000], [1, 20, 'mers', 2002], [2, 30, 'nisan', 2015]]
names = ['Index', 'speed', 'mark_car', 'year']
data = pd.DataFrame(a, columns=names)
data2 = DataFrame(a, columns=names)


In [26]:
list(data.columns) == data2.columns[1:]

True

In [27]:
data2.shape == data.shape

True

In [28]:
data2.shape

(3, 4)

In [29]:
data2['speed'] == data['speed']

0    True
1    True
2    True
Name: speed, dtype: bool

In [30]:
data2.head(1)

+-------+-------+-------+----------+------+
| index | Index | speed | mark_car | year |
+-------+-------+-------+----------+------+
|   0   |   0   |   10  |   bmv    | 2000 |
+-------+-------+-------+----------+------+

In [31]:
data2['1'] = [1] * 3

In [32]:
print(data2.head())

+-------+-------+-------+----------+------+---+
| index | Index | speed | mark_car | year | 1 |
+-------+-------+-------+----------+------+---+
|   0   |   0   |   10  |   bmv    | 2000 | 1 |
|   1   |   1   |   20  |   mers   | 2002 | 1 |
|   2   |   2   |   30  |  nisan   | 2015 | 1 |
+-------+-------+-------+----------+------+---+


In [33]:
data

,Index,speed,mark_car,year
0,0,10,bmv,2000
1,1,20,mers,2002
2,2,30,nisan,2015


In [34]:
data.head(1)

,Index,speed,mark_car,year
0,0,10,bmv,2000


In [35]:
data2

+-------+-------+-------+----------+------+---+
| index | Index | speed | mark_car | year | 1 |
+-------+-------+-------+----------+------+---+
|   0   |   0   |   10  |   bmv    | 2000 | 1 |
|   1   |   1   |   20  |   mers   | 2002 | 1 |
|   2   |   2   |   30  |  nisan   | 2015 | 1 |
+-------+-------+-------+----------+------+---+

In [36]:
print(data2['speed'] + data2['speed'], '\n')
print(data['speed'] + data['speed'])

[20, 40, 60] 

0    20
1    40
2    60
Name: speed, dtype: int64


In [37]:
data2.data

[['index', 'Index', 'speed', 'mark_car', 'year', '1'], [0, 0, 10, 'bmv', 2000, 1], [1, 1, 20, 'mers', 2002, 1], [2, 2, 30, 'nisan', 2015, 1]]

In [38]:
data2.columns

['index', 'Index', 'speed', 'mark_car', 'year', '1']

In [39]:
data2['speed']

[10, 20, 30]

In [40]:
m = mpd.read_csv('test.csv', delimiter= ',')
m.head()

+-------+--------+-----+------+---------+---------+------------+------------+-----------+----------+---------+----------------+---------------+--------+
| index | season |  yr | mnth | holiday | weekday | workingday | weathersit |    temp   |  atemp   |   hum   | windspeed(mph) | windspeed(ms) |  cnt   |
+-------+--------+-----+------+---------+---------+------------+------------+-----------+----------+---------+----------------+---------------+--------+
|   0   |  1.0   | 0.0 | 1.0  |   0.0   |   6.0   |    0.0     |    2.0     | 14.110847 | 18.18125 | 80.5833 |   10.749882    | 4.80549038891 | 985.0  |
|   1   |  1.0   | 0.0 | 1.0  |   0.0   |   0.0   |    0.0     |    2.0     | 14.902598 | 17.68695 | 69.6087 |   16.652113    | 7.44394859186 | 801.0  |
|   2   |  1.0   | 0.0 | 1.0  |   0.0   |   1.0   |    1.0     |    1.0     |  8.050924 | 9.47025  | 43.7273 |   16.636703    | 7.43705990165 | 1349.0 |
|   3   |  1.0   | 0.0 | 1.0  |   0.0   |   2.0   |    1.0     |    1.0     |    8

In [41]:
b = pd.read_csv('test.csv')
a = read_csv('test.csv')
b.head(5)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed(mph),windspeed(ms),cnt
0,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,4.805490,985
1,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,7.443949,801
2,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,7.437060,1349
3,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,4.800998,1562
4,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,5.597810,1600


In [42]:
a.head(3)

+-------+--------+-----+------+---------+---------+------------+------------+--------+--------+--------+----------------+---------------+--------+
| index | season |  yr | mnth | holiday | weekday | workingday | weathersit |  temp  | atemp  |  hum   | windspeed(mph) | windspeed(ms) |  cnt   |
+-------+--------+-----+------+---------+---------+------------+------------+--------+--------+--------+----------------+---------------+--------+
|   0   |  1.0   | 0.0 | 1.0  |   0.0   |   6.0   |    0.0     |    2.0     | 14.111 | 18.181 | 80.583 |     10.75      |     4.805     | 985.0  |
|   1   |  1.0   | 0.0 | 1.0  |   0.0   |   0.0   |    0.0     |    2.0     | 14.903 | 17.687 | 69.609 |     16.652     |     7.444     | 801.0  |
|   2   |  1.0   | 0.0 | 1.0  |   0.0   |   1.0   |    1.0     |    1.0     | 8.051  |  9.47  | 43.727 |     16.637     |     7.437     | 1349.0 |
+-------+--------+-----+------+---------+---------+------------+------------+--------+--------+--------+--------------

In [43]:
data2

+-------+-------+-------+----------+------+---+
| index | Index | speed | mark_car | year | 1 |
+-------+-------+-------+----------+------+---+
|   0   |   0   |   10  |   bmv    | 2000 | 1 |
|   1   |   1   |   20  |   mers   | 2002 | 1 |
|   2   |   2   |   30  |  nisan   | 2015 | 1 |
+-------+-------+-------+----------+------+---+

In [44]:
data2['dcss'] = [4, 5, 6]
data2.data

[['index', 'Index', 'speed', 'mark_car', 'year', '1', 'dcss'], [0, 0, 10, 'bmv', 2000, 1, 4], [1, 1, 20, 'mers', 2002, 1, 5], [2, 2, 30, 'nisan', 2015, 1, 6]]

In [45]:
data2.head(1)

+-------+-------+-------+----------+------+---+------+
| index | Index | speed | mark_car | year | 1 | dcss |
+-------+-------+-------+----------+------+---+------+
|   0   |   0   |   10  |   bmv    | 2000 | 1 |  4   |
+-------+-------+-------+----------+------+---+------+

In [46]:
data2['year'] = [1, 2, 3]
data2

+-------+-------+-------+----------+------+---+------+
| index | Index | speed | mark_car | year | 1 | dcss |
+-------+-------+-------+----------+------+---+------+
|   0   |   0   |   10  |   bmv    |  1   | 1 |  4   |
|   1   |   1   |   20  |   mers   |  2   | 1 |  5   |
|   2   |   2   |   30  |  nisan   |  3   | 1 |  6   |
+-------+-------+-------+----------+------+---+------+

In [47]:
data2.shape

(3, 6)

In [426]:
data2

+-------+-------+-------+----------+------+---+------+
| index | Index | speed | mark_car | year | 1 | dcss |
+-------+-------+-------+----------+------+---+------+
|   0   |   0   |   10  |   bmv    | 2000 | 1 |  4   |
|   1   |   1   |   20  |   mers   | 2002 | 1 |  5   |
|   2   |   2   |   30  |  nisan   | 2015 | 1 |  6   |
+-------+-------+-------+----------+------+---+------+